In [13]:
import $file.common
import common._
import cats.data._, cats._, cats.implicits._, cats.evidence._
import doobie.implicits._
import doobie.util.fragment._

import $file.$     

import common._

import cats.data._, cats._, cats.implicits._, cats.evidence._

import doobie.implicits._

import doobie.util.fragment._

# Variation 6a. Finally, QUEΛ

We will now review the tagless-final solution to the query avalanche problem, previously solved using Quoted DSLs in `quill`. This is described in the following [paper](http://okmij.org/ftp/meta-programming/quel.pdf) by Suzuki, Kiselyov (the father of tagless-final) and Kameyama, which introduces the tagless-final [`QUEΛ`](http://logic.cs.tsukuba.ac.jp/~ken/quel/) DSL: 

![](images/quelpaper.png)

The tagless-final approach resembles MTL in the use of type classes to define the syntax of the DSL. And the use of this custom syntax, instead of the Scala AST, makes it different from Quoted DSLs at the same time. But tagless-final departs from MTL in one important aspect: tagless-final type classes are not parameterised by a generic _computation type_, but by a generic _representation_. We will see soon what this means. 

Let's start by recalling the in-memory query `largeCapitals` (the quoted version in the `quill` example - [variation 5](Variation5.Quill.ipynb)), which serves as a sort of specification for our DSL:

In [14]:
case class Country(
    code: String, 
    name: String, 
    capital: Option[Int])

case class City(
    id: Int, 
    name: String, 
    countryCode: String, 
    population: Int)

case class World(
    countries: Map[String, Country],
    cities: Map[Int, City]){
    
    val allCountries: List[Country] = 
        countries.values.toList
    val allCities: List[City] = 
        cities.values.toList
}

def largeCapitals(implicit world: World): List[(String, String)] =
    for {
        country <- world.allCountries
        city <- world.allCities
        if country.capital.exists(_ == city.id)
        if city.population > 8000000
    } yield (city.name, country.name)

defined class Country
defined class City
defined class World
defined function largeCapitals

### Syntax and type system

The `largeCapitals` query is made from two major kinds of expressions: those pertaining to the _world_ database, and those related to the construction of queries themselves (i.e. independent of the particular database model). Beginning with the former ones, we encounter: 
* `world.allCountries` and `world.allCities`, which represent list expressions (`List[Country]` and `List[City]`, in particular)
* `country.capital`, which represents an optional value `Option[Int]`
* `country.name`, `city.name` and `city.population`, which represent basic values of type `String` and `Int`


The syntax and type system of a mini-language to talk about countries, cities and their attributes is encoded in the following type classes:

In [15]:
// Case classes, as before

case class Country(code: String, name: String, capital: Option[Int])
case class City(id: Int, name: String, countryCode: String, population: Int)

// Data models as type constructor (representation) classes

trait CityModel[Repr[_]]{
    def cityId(city: Repr[City]): Repr[Int]
    def cityName(city: Repr[City]): Repr[String]
    def cityCountry(country: Repr[City]): Repr[String]
    def cityPopulation(city: Repr[City]): Repr[Int]
}

trait CountryModel[Repr[_]]{
    def countryName(country: Repr[Country]): Repr[String]
    def countryCapital(country: Repr[Country]): Repr[Option[Int]]
}

trait WorldModel[Repr[_]] extends CityModel[Repr] with CountryModel[Repr]{
    def allCountries: Repr[List[Country]]
    def allCities: Repr[List[City]]
}

defined class Country
defined class City
defined trait CityModel
defined trait CountryModel
defined trait WorldModel

The major similarities and differences with the type classes of MTL-based repositories are the following ones: 
* The type constructor is named by convention `Repr[_]`, instead of `F[_]`. This is a minor notational difference, but serves us well to emphasise that we are talking here about _representation_ classes, not _computation_ classes. For instance, the MTL-based constant `def allCities: F[City]` is implemented in terms of a computation that will _return_ a stream or list of cities when executed. On the contrary, the tagless-final `def allCities: Repr[List[City]]` need not be implemented in terms of computations at all. It could be, and we will see an example in the standard semantics used for unit testing, but, it could also be a non-computational, intermediate data structure, as we will see with the non-standard semantics used to generate optimal SQL queries.
* If `Repr[_]` is a representation, what are then its arguments in the different signatures? We refer to the types `Int`, `String`, `Option[Int]`, `List[City]` and `List[Country]` (which even refer to the case classes `City` and `Country` used in MTL repos, `quill` and in-memory models). In general, arguments of the type contructor `Repr[_]` serve the purpose of defining the __type system__ of the language. This component of our language complements the syntax of the DSL and allow us to avoid writing non-sensical expressios such as `cityName(c)`, where `c: Country`. Moreover, referring to the domain case classes `City` and `Country`, as well as to the Scala standard types `Int`, `String` and `List` is optional. We will see that this is most convenient when giving the standard semantics, but we may have well chosen abstract type members for this purpose. Typically, in non-computational representations of the language, these types play the roles of _phantom types_.

Let's come back to the `largeCapitals` query and analyse its general querying expressions. As we can see, our DSL must give us the possibility of writing: 
* _Integer expressions_, e.g. the constant `8000000`, and the comparison `city.population > 8000000`
* _Equality expressions_, e.g. `_ == city.id`
* _Algebraic data type (ADT) expressions_, concerning products, e.g. `(city.name, country.name)` and optional values, e.g. `...exists(...)`
* _Comprehension expressions_, i.e. `for { ... <- ...; if ....; ... } yield ...`, which are made from the higher-order functions `flatMap`, `filter` and `map`.

The `QUEΛ` language deals with these later kinds of expressions, i.e. it is a general query language, deprived of any data model construct. The version that will be shown below differs in a couple of things: first, it is first-order, i.e. no lambda expressions (this in line with the language [`SQUR`](http://okmij.org/ftp/meta-programming/#SQUR), a subsequent development of the authors); second, we add a couple of constructs related to tuples and optional values; last, its present version is incomplete, we just focus on what we need to write our sample query. We present the language as a composition of sub-languages, as it's typically done in the tagless-final style:

In [16]:
trait BaseExpr[Repr[_]]{
    
    def bool(b: Boolean): Repr[Boolean]
    def int(i: Int): Repr[Int]
    def str(s: String): Repr[String]
    
    def ===(a1: Repr[Int], a2: Repr[Int]): Repr[Boolean]
    def >(i1: Repr[Int], i2: Repr[Int]): Repr[Boolean]
    // lots of relational operators more ...
}

trait ADTExpr[Repr[_]]{
    
    def tuple2[A, B](a: Repr[A], b: Repr[B]): Repr[(A, B)]
    
    def none[A]: Repr[Option[A]]
    def some[A](a: Repr[A]): Repr[Option[A]]
    def exists[A](o: Repr[Option[A]])(
        cond: Repr[A] => Repr[Boolean]): Repr[Boolean]
}

trait MultisetExpr[Repr[_]]{
    def from[A, B](q: Repr[List[A]])(f: Repr[A] => Repr[List[B]]): Repr[List[B]]
    def where[A](cond: Repr[Boolean])(q: Repr[List[A]]): Repr[List[A]]
    def select[A](a: Repr[A]): Repr[List[A]]
    // omitted: multiset union, empty test, ... 
}

trait QUEΛ[Repr[_]] extends 
    BaseExpr[Repr] with 
    ADTExpr[Repr] with 
    MultisetExpr[Repr]

defined trait BaseExpr
defined trait ADTExpr
defined trait MultisetExpr
defined trait QUEΛ

Now we are ready to write the `largeCapitals` query!

In [17]:
def largeCapitals[Repr[_]](implicit 
        Q: QUEΛ[Repr], W: WorldModel[Repr]): Repr[List[(String, String)]] = 
    Q.from(W.allCountries){ country =>
        Q.from(W.allCities){ city => 
            Q.where(Q.exists(W.countryCapital(country))(Q.===(_, W.cityId(city)))){
                Q.where(Q.>(W.cityPopulation(city), Q.int(8000000))){
                    Q.select(Q.tuple2(W.cityName(city), W.countryName(country)))
                }
            }
        }
    }

defined function largeCapitals

Ooops, pretty ugly. We need some sugar to avoid referencing the type class instances, `Q` and `W`, and, specially, to profit from Scala for-comprehensions:

In [18]:
object WorldModelSyntax{
    def allCountries[Repr[_]](implicit W: WorldModel[Repr]) = 
        W.allCountries
    def allCities[Repr[_]](implicit W: WorldModel[Repr]) = 
        W.allCities
    
    implicit class CountryFields[Repr[_]](country: Repr[Country])(implicit W: WorldModel[Repr]){
        def name = W.countryName(country)
        def capital = W.countryCapital(country)
    }
    
    implicit class CityFields[Repr[_]](city: Repr[City])(implicit W: WorldModel[Repr]){
        def id = W.cityId(city)        
        def name = W.cityName(city)        
        def population = W.cityPopulation(city)
    }
}

import WorldModelSyntax._

object QUEΛSyntax{
    implicit class ComprehensionOps[Repr[_], A](la: Repr[List[A]])(implicit Q: QUEΛ[Repr]){
        def flatMap[B](cont: Repr[A] => Repr[List[B]]): Repr[List[B]] = 
            Q.from(la)(cont)
        def map[B](f: Repr[A] => Repr[B]): Repr[List[B]] = 
            Q.from(la)(a => Q.select(f(a)))
        def withFilter(p: Repr[A] => Repr[Boolean]): Repr[List[A]] = 
            Q.from(la)(a => Q.where(p(a))(Q.select(a)))
    }

    implicit class BinOps[Repr[_]](a1: Repr[Int])(implicit Q: QUEΛ[Repr]){
        def ===(a2: Repr[Int]): Repr[Boolean] = Q.===(a1, a2)
        def >(a2: Repr[Int]): Repr[Boolean] = Q.>(a1, a2)
    }

    implicit def Tuple2QUEΛ[Repr[_], A, B](t2: (Repr[A], Repr[B]))(implicit Q: QUEΛ[Repr]): Repr[(A, B)] = 
        Q.tuple2(t2._1, t2._2)

    implicit class OptionOps[Repr[_], A](o: Repr[Option[A]])(implicit Q: QUEΛ[Repr]){
        def exists(cond: Repr[A] => Repr[Boolean]): Repr[Boolean] = 
            Q.exists(o)(cond)
    }

    implicit def IntQUEΛ[Repr[_]](i: Int)(implicit Q: QUEΛ[Repr]): Repr[Int] = 
        Q.int(i)
}

import QUEΛSyntax._

defined object WorldModelSyntax
import WorldModelSyntax._


defined object QUEΛSyntax
import QUEΛSyntax._

Now, believe it or not, we can write our query using almost the very same syntax than for in-memory queries:

In [19]:
def largeCapitals[Repr[_]: QUEΛ: WorldModel]: Repr[List[(String, String)]] = 
    for {
        country <- allCountries
        city <- allCities
        if country.capital.exists(_ === city.id)
        if city.population > 8000000
    } yield (city.name, country.name)

defined function largeCapitals

### Standard semantics

Ok, but what do QUEΛ expressions _mean_? The answer is provided by the _standard_ semantics, which is typically given by the `Id[_]` representation. This means that `Repr[Int] == Int`, `Repr[List[City]] == List[City]`, and so forth, i.e. we give meanings to expressions in terms of the types chosen to define the type system of the language. Thus, the standard meaning of `QUEΛ` expressions is given by in-memory standard Scala types. Similarly, the standard meaning for `WorldModel` expressions is given by in-memory case classes. 

In [20]:
implicit object StdQUEΛ extends QUEΛ[Id]{
    // BASE TYPES

    def bool(b: Boolean): Boolean = b
    def int(i: Int): Int = i
    def str(s: String): String = s
    
    def ===(a1: Int, a2: Int): Boolean = a1 == a2
    def >(i1: Int, i2: Int): Boolean = i1 > i2

    // ADTs

    def tuple2[A, B](a: A, b: B): (A, B) = (a,b)

    def none[A]: Option[A] = None
    def some[A](a: A): Option[A] = Some(a)
    def exists[A](a: Option[A])(cond: A => Boolean): Boolean = 
        a.flatMap(cond andThen Option.apply).getOrElse(false)

    // COMPREHENSIONS

    def from[A, B](q: List[A])(f: A => List[B]): List[B] = q.flatMap(f)
    def select[A](a: A): List[A] = List(a)
    def where[A](cond: Boolean)(q: List[A]): List[A] = if (cond) q else List()
}

defined object StdQUEΛ

The standard semantics of `WorldModel` given in terms of `Id[_]` requires a `World` parameter. Alternatively, we may have given the standard semantics in terms of `StateT[List, World, _]`, as we did for the MTL-based repos, but we preferred `Id[_]` in order for the `QUEΛ` semantics to be more simpler. 

In [21]:
case class World(countries: Map[String, Country], cities: Map[Int, City])

implicit def IdWorldModel(implicit W: World): WorldModel[Id] = 
    new WorldModel[Id]{
        // World
        def allCountries: List[Country] = W.countries.values.toList
        def allCities: List[City] = W.cities.values.toList
        
        // Cities
        def cityId(city: City): Int = city.id
        def cityName(city: City): String = city.name
        def cityCountry(city: City): String = city.countryCode
        def cityPopulation(city: City): Int = city.population

        // Countries
        def countryCode(country: Country): String = country.code
        def countryName(country: Country): String = country.name
        def countryCapital(country: Country): Option[Int] = country.capital
    }

defined class World
defined function IdWorldModel

Using this standard semantics, we can do unit testing, much in the same way as before:

In [22]:
import org.scalatest._

object LargeCapitalsSpec extends FlatSpec with Matchers{
    
    implicit val smallWorld: World =         
        World(
            Map("ES" -> Country("ES","Spain",Some(0)),
                "USA" -> Country("USA", "United States", Some(1)),
                "UK" -> Country("UK", "United Kingdom", Some(2)),
                "UNK" -> Country("UNK", "Unknown", None)),
            Map(0->City(0,"Madrid","ES",9000000),
                1->City(1,"Washington", "USA", 10000000),
                2->City(2,"London", "UK", 500000)))    

    "large capitals" should "be right" in {
        largeCapitals[Id].toSet shouldBe 
            Set(("Madrid", "Spain"), ("Washington", "United States"))
    }
}

import org.scalatest._


defined object LargeCapitalsSpec

In [23]:
run(LargeCapitalsSpec)

cmd21$Helper$LargeCapitalsSpec:
large capitals
- should be right


All is good, but, up to now, we haven't done anything that we couldn't do with MTL-based repos! So, let's move to more [interesting semantics](Variation6b.QUEΛ.ipynb).